# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [35]:
import os
from glob import glob

# Write your code below.

# Loading the data into pandas dataframe format
ft_dir = os.getenv("PRICE_DATA")
ft_glob = glob(os.path.join(ft_dir, '**/*.parquet'), recursive = True)
ft_glob = [f for f in ft_glob if os.path.isfile(f)]
df = dd.read_parquet(ft_glob)
df.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463039,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142937,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652866,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219177,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237450,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [36]:
# Write your code below.

df['Date'] = dd.to_datetime(df['Date'])
df['Close_lag'] = df['Close'].shift(1)
df['Adj Close_lag'] = df['Adj Close'].shift(1)
df['returns'] = (df['Adj Close'] / df['Adj Close_lag']) - 1
df['hi_lo_range'] = df['High'] - df['Low']

dd_feat = df

dd_feat.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463039,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744
A,2000-01-04 00:00:00+00:00,40.142937,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.463039,-0.076389,2.950645
A,2000-01-05 00:00:00+00:00,37.652866,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.142937,-0.062030,4.425964
A,2000-01-06 00:00:00+00:00,36.219177,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.652866,-0.038076,2.771820
A,2000-01-07 00:00:00+00:00,39.237450,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.219177,0.083334,4.962444


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [37]:
# Write your code below.

# Convert to pandas DataFrame
df = dd_feat.compute()

# Add a 10-day rolling average for returns
df['rolling_avg_return_10d'] = df['returns'].rolling(window=10).mean()

# Display the final DataFrame
print(df.head(15))


Price                       Date  Adj Close      Close       High        Low  \
ticker                                                                         
A      2000-01-03 00:00:00+00:00  43.463039  51.502148  56.464592  48.193848   
A      2000-01-04 00:00:00+00:00  40.142937  47.567955  49.266811  46.316166   
A      2000-01-05 00:00:00+00:00  37.652866  44.617310  47.567955  43.141991   
A      2000-01-06 00:00:00+00:00  36.219177  42.918453  44.349072  41.577251   
A      2000-01-07 00:00:00+00:00  39.237450  46.494991  47.165592  42.203148   
A      2000-01-10 00:00:00+00:00  41.614357  49.311516  49.803291  48.327969   
A      2000-01-11 00:00:00+00:00  41.048416  48.640915  49.311516  47.523247   
A      2000-01-12 00:00:00+00:00  40.218395  47.657368  48.640915  45.824390   
A      2000-01-13 00:00:00+00:00  40.822052  48.372677  49.937412  47.210300   
A      2000-01-14 00:00:00+00:00  41.274784  48.909157  49.624462  47.925610   
A      2000-01-18 00:00:00+00:00  43.161

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

Not actually, it was not necessary. It would have been better to perform this operation in Dask, as Dask is designed for handling larger-than-memory datasets and can efficiently compute rolling averages without needing to load all data into memory, maintaining better performance and scalability.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.